In [25]:
import numpy as np
import matplotlib.pyplot as plt

datos = np.loadtxt('datos_observacionales.dat')

t = datos[:,0]
x = datos[:,1]
y = datos[:,2]
z = datos[:,3]
p =np.random.random()*30
y = np.random.random()

def dx(sigma,y,x):
    return sigma*(y-x)
def dy (x,p,z,y):
    return x*(p-z)*y
def dz(x,y,z,b):
    return x*y - b*z

def moveX (sigma,y,x):
    xnew= x + dx(sigma,y,x)
    return xnew
def moveY (x,p,z,y):
    ynew = y + dy(x,p,z,y)
    return ynew
def moveZ(x,y,z,b):
    znew = z + dz(x,y,z,b)
    return znew

def model (x,y,z,p,b,sigma):
    return x*sigma + y*p + b*p

def loglikelihood(x_obs, y_obs, z, p, b,sigma):
    d = model(x_obs, y, z,p,b,sigma)
    d = d/sigma
    d = -0.5 * np.sum(d**2)
    return d


def logprior(b, p,sigma):
    p = -np.inf
    if sigma < 30 and b< 30 and p < 30 :
        p = 0.0
    return p

N = 50000

lista_b = [np.random.random()]
lista_p = [np.random.random()]
lista_sigma = [np.random.random()]

sigma_delta_p = 1.0
sigma_delta_b = 1.0
sigma_delta_sigma = 1.0
logposterior =[]
for i in range(1,N):
    propuesta_p  = lista_p[i-1] + np.random.normal(loc=0.0, scale=sigma_delta_p)
    propuesta_b  = lista_b[i-1] + np.random.normal(loc=0.0, scale=sigma_delta_b)
    propuesta_sigma = lista_sigma[i-1] + np.random.normal(loc=0.0, scale= sigma_delta_sigma)
    
    logposterior_viejo = loglikelihood(x, y, z, lista_p[i-1], lista_b[i-1], lista_sigma[i-1]) + logprior(lista_p[i-1], lista_b[i-1],lista_sigma[i-1])
    logposterior_nuevo = loglikelihood(x, y, z, propuesta_p, propuesta_b, propuesta_sigma) + logprior(propuesta_p, propuesta_b,propuesta_sigma)
  
    r = min(1,np.exp(logposterior_nuevo-logposterior_viejo))
    alpha = np.random.random()
    if(alpha<r):
        lista_p.append(propuesta_p)
        lista_b.append(propuesta_b)
        lista_sigma.append(propuesta_sigma)
        
        logposterior.append(logposterior_nuevo)
    else:
        lista_p.append(lista_p[i-1])
        lista_b.append(lista_b[i-1])
        lista_sigma.append(lista_sigma[i-1])
        logposterior.append(logposterior_viejo)
        
lista_p = np.array(lista_p)
lista_b = np.array(lista_b)
lista_sigma= np.array(lista_sigma)

logposterior = np.array(logposterior)

pp = np.mean(lista_p)
bb = np.mean(lista_b)
sigmaa = np.mean(lista_sigma)

In [34]:
def leapfrog(x,y,z, sigma, delta_t=1E-1, niter=5):
    x_new = x
    y_new = y
    z_new = z
    for i in range(niter):
        x_new = moveX(sigmaa,y,x)
        y_new = moveY (x,pp,z,y)
        z_new = moveZ(x,y,z,bb)
    return x_new, y_new, z_new


def MCMC(nsteps):
    x = np.zeros(nsteps)
    y = np.zeros(nsteps)
    z = np.zeros(nsteps)
    x[0] = np.random.normal(0,1)
    y[0] = np.random.normal(0,1)
    z[0] = np.random.normal(0,1)
    sigma = 0.1
    for i in range(1,nsteps):
        x[i] = np.random.normal(0,1)
        x_new, y_new,z_new = leapfrog(x[i-1],y[i-1],z[i-1], sigma) 

        E_new = model(x_new, y_new, z_new,pp,bb,sigmaa ) 
        E_old = model(x[i-1], y[i-1],z[i-1],pp,bb, sigmaa)
        alpha = min(1.0,np.exp(-(E_new - E_old)))
        
        beta = np.random.random()
        if beta < alpha:
            x[i] = x_new
            y[i] = y_new
            z[i] = z_new
        else:
            x[i] = x[i-1]
            y[i] = y[i-1]
            z[i] = z[i-1]
       
    
MCMC(1000)

/srv/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in exp


IndentationError: expected an indented block (<ipython-input-21-0f3cc5071291>, line 22)